# Unupervise Learning - Week4 Assignment - Part 2

## Limitation(s) of sklearn’s non-negative matrix factorization library. 

### 0. Instruction:

1. Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE. You should use sklearn library. 

2. Discuss the results and why they did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it?

### Question 1. NMF-based method

The data is loacted in the same folder of the nootbook. Also, it is uploaded in the following repository:

https://github.com/alex-cage/Courseshare/tree/1327aadfd1e75cb946896e55b55aa3c9f5b6b87c/SupervisedLearning/Week4

In [1]:
########################################################################################
# Import libraries
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import root_mean_squared_error
from scipy.sparse import coo_matrix

########################################################################################
# load csvs
train_df = pd.read_csv("train.csv")      
test_df = pd.read_csv("test.csv")         
users_df = pd.read_csv("users.csv")      
movies_df = pd.read_csv("movies.csv")    

########################################################################################
# Map User and Movie IDs to Matrix Indices
all_users = list(users_df['uID'])         
all_movies = list(movies_df['mID'])      

uid2idx = {uid: idx for idx, uid in enumerate(all_users)}  
mid2idx = {mid: idx for idx, mid in enumerate(all_movies)} 

########################################################################################
#  Build the Sparse Ratings Matrix for Training
ind_user_train = [uid2idx[u] for u in train_df['uID']]    
ind_movie_train = [mid2idx[m] for m in train_df['mID']]    
rating_train = list(train_df['rating'])                   

# Construct sparse user-movie rating matrix (users x movies)
Mr_sparse = coo_matrix((rating_train, (ind_user_train, ind_movie_train)),
                       shape=(len(all_users), len(all_movies)))

########################################################################################
#  Train the NMF Model
nmf_model = NMF(n_components=20,          
                random_state=42,
                init='random',
                max_iter=500)
W = nmf_model.fit_transform(Mr_sparse)    
H = nmf_model.components_                 

########################################################################################
# Pedict Ratings 
Mr_pred = np.dot(W, H)  

########################################################################################
# Prepare Test Set Indices and Extract Ground Truth
ind_user_test = [uid2idx[u] for u in test_df['uID']]       # row indices for test
ind_movie_test = [mid2idx[m] for m in test_df['mID']]      # column indices for test
rating_test = list(test_df['rating'])                      # true ratings

# Predict the rating for each test (user, movie) pair
predicted_ratings = [Mr_pred[u_idx, m_idx] for u_idx, m_idx in zip(ind_user_test, ind_movie_test)]

########################################################################################
# Result
rmse = root_mean_squared_error(rating_test, predicted_ratings)
print(f"RMSE is: {rmse:.2f}")


RMSE is: 2.85


### Question 2. Conclusion

* Discuss the results and why sklearn's non-negative matrix facorization library did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it? <br>
**Answer**:<br>
1. The based line RMSE is 1.26 ($Y_p=3$), while the NMF method provides a RMSE of 2.85, suggesting the NMF method doesn't give a good fit.<br>
2. The main drawbacks of the NMF method in this case are:<br>
(1) The NMF method doesn't focus on the user or movie features, such as age, region, genres. Instead, it focuses on the matrix, which reduces the information utilized.<br>
(2) The data is sparse, and the method puts more efforts to estimate the zeros, which could be overfitting.<br>
4. Countering the two disadvantages, similarity-based models or the alternating least squares method (that can handle the sparcity) could improve the prediction.